In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn
import os

In [4]:
from tensorboard.plugins.hparams import api as hp

In [5]:
#image augmentation, batch normalization, learning rate, initialization
ia_list = [False, True]
bn_list = [False, True]
init_list = ['RandomNormal_0.01std',
             'glorot_normal',
             'he_normal']
lr_list = [0.001, 0.01, 0.03]

In [6]:
hp_ia = hp.HParam(name='ia',
                  domain=hp.Discrete(ia_list),
                  display_name='Image Augmentation')

hp_bn = hp.HParam(name='bn',
                  domain=hp.Discrete(bn_list),
                  display_name='Batch Normalization')

hp_init = hp.HParam(name='init',
                  domain=hp.Discrete(init_list),
                  display_name='Weight Initialization')

hp_lr = hp.HParam(name='lr',
                  domain=hp.Discrete(lr_list),
                  display_name='Learning Rate')

hp_metric = hp.Metric('accuracy', display_name='Accuracy')

1. hp.HParam 
- name: hyper parameter id
- domain: value of the parameter (need to use hp.Discrete or hp.RealInterval for conversion)
- display_name: Name displayed on Tensorboard
2. hp.Metric
- name: hyper parameter id
- display_name: Name displayed on Tensorboard
3. hp.Discrete
- fixed value for the parameters, such as hp.Discrete([0,1,2]) has value 0,1,2
4. hp.RealInterval
- range value for the parameters, such as hp.RealInterval(0.1, 0.2) has value between 0.1~0.2

In [7]:
log_dirs = os.path.join('lab7-logs-hparams', 'hparam_tuning')
root_logdir_writer = tf.summary.create_file_writer(log_dirs)
with root_logdir_writer.as_default():
    hp.hparams_config(hparams=[hp_ia, hp_bn, hp_init, hp_lr], metrics=[hp_metric])

In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
batch_size = 64  # 批次大小
# 將train Data重新分成1:9等分，分別分給valid data, train data
train_split, valid_split = ['train[:90%]', 'train[90%:]']

# 取得訓練數據
train_data_noaug, info = tfds.load("cifar10", split=train_split, with_info=True)
train_data_aug = tfds.load("cifar10", split=train_split)
# 取得驗證數據
valid_data = tfds.load("cifar10", split=valid_split)

train_num = int(info.splits['train'].num_examples / 5) * 4  # 訓練資料數量

train_data_noaug = train_data_noaug.shuffle(train_num)  # 打散資料集
# 載入預處理「 parse_aug_fn」function，cpu數量為自動調整模式
train_data_noaug = train_data_noaug.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
train_data_noaug = train_data_noaug.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

train_data_aug = train_data_aug.shuffle(train_num)  # 打散資料集
# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
train_data_aug = train_data_aug.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
train_data_aug = train_data_aug.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [20]:
class HyperparameterCallBack(tf.keras.callbacks.Callback):
    # 類別創建時調用
    def __init__(self, log_dir, hparams):
        super(HyperparameterCallBack, self).__init__()
        self.log_dir = log_dir
        self.hparams = hparams
        self.best_accuracy = 0
        self.writer = None
    
    # 訓練開始前調用，建立一個紀錄檔，並寫入本次實驗的超參數
    def on_train_begin(self, logs=None):
        self.writer = tf.summary.create_file_writer(self.log_dir)
        #記錄超參數
        with self.writer.as_default():
            hp.hparams(self.hparams)
            
    # 每一個Epoch結束後調用，每一個訓練Epoch結束後，如果模型有進步會更新準確率(Accuracy)
    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get('val_categorical_accuracy')
        if current_accuracy >self.best_accuracy:
            self.best_accuracy = current_accuracy
            
    # 訓練結束時調用，將最佳的準確率存入紀錄檔中，並寫入[訓練完成]事件
    def on_train_end(self, logs=None):
        with self.writer.as_default():
            tf.summary.scalar('accuracy', self.best_accuracy, step=0)

In [21]:
def train_test_model(logs_dir, hparams):
    """
    logs_dir:傳入目前執行的任務log檔的位置
    hparams:傳入超參數
    """
    # 指派網路模型初始化的方法
    #權重初始化: 初始化使用[Glorot], [He] 或 [標準差為 0.01 的常態分佈]
    if hparams[hp_init] == "glorot_normal":
        init = initializers.glorot_normal()
    elif hparams[hp_init] == "he_normal":
        init = initializers.he_normal()
    else:
        init = initializers.RandomNormal(0, 0.01)
        
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(64, (3, 3), kernel_initializer=init)(inputs)
    #Batch normalization 層: 選擇[加]或[不加] Batch Normalization 層
    if hparams[hp_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, (3, 3), kernel_initializer=init)(x)
    #Batch normalization 層: 選擇[加]或[不加] Batch Normalization 層
    if hparams[hp_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(256, (3, 3), kernel_initializer=init)(x)
    #Batch normalization 層: 選擇[加]或[不加] Batch Normalization 層
    if hparams[hp_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, (3, 3), kernel_initializer=init)(x)
    #Batch normalization 層: 選擇[加]或[不加] Batch Normalization 層
    if hparams[hp_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(64, (3, 3), kernel_initializer=init)(x)
    #Batch normalization 層: 選擇[加]或[不加] Batch Normalization 層
    if hparams[hp_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, kernel_initializer=init)(x)
    #Batch normalization 層: 選擇[加]或[不加] Batch Normalization 層
    if hparams[hp_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    # 建立網路模型(將輸入到輸出所有經過的網路層連接起來)
    model = keras.Model(inputs, outputs, name='model')
    
    # 儲存訓練記錄檔
    model_tb = keras.callbacks.TensorBoard(log_dir=logs_dir, write_graph=False)
    
    # 儲存最好的網路模型權重
    model_mckp = keras.callbacks.ModelCheckpoint(logs_dir +'/Best-model.h5',
                                                 monitor='val_categorical_accuracy',
                                                 save_best_only=True,
                                                 mode='max')
    
    # 設定停止訓練的條件(當Accuracy超過30迭代沒有上升的話訓練會終止)
    model_els = keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy',
                                              min_delta=0,
                                              patience=30,
                                              mode='max')
    
    # 客製化超參數回調函式，紀錄訓練模型的超參數和指標(準確率)
    model_hparam = HyperparameterCallBack(logs_dir + 'validation', hparams)
    
    # 設定訓練使用的優化器、損失函數和指標函數
    # 優化器學習率為超參數：0.001、0.01或0.03
    model.compile(keras.optimizers.Adam(hparams[hp_lr]),
                  loss = keras.losses.CategoricalCrossentropy(),
                  metrics = [keras.metrics.CategoricalAccuracy()])
    
    # 超參數：使用「經過影像增強的數據」或「不經過影像增強的數據」訓練網路
    if hparams[hp_ia]:
        history = model.fit(train_data_aug,
                            epochs=100,
                            validation_data=valid_data,
                            callbacks=[model_tb, model_mckp, model_els, model_hparam])
        
    else:
        history = model.fit(train_data_noaug,
                            epochs=100,
                            validation_data=valid_data,
                            callbacks=[model_tb, model_mckp, model_els, model_hparam])

In [ ]:
session_id = 1 #訓練任務的id
# 儲放紀錄檔的位置
logs_dir = os.path.join('lab7-logs-hparams','run-{}')
for ia in ia_list:
    for bn in bn_list:
        for init in init_list:
            for lr in lr_list:
                # 顯示目前訓練任務id
                print('--- Running training session {}'.format(session_id))
                # 設定本次訓練的超參數
                hparams = {hp_ia: ia, hp_bn: bn, hp_init: init, hp_lr: lr}
                # 建立、編譯及訓練網路模型
                train_test_model(logs_dir.format(session_id), hparams)
                session_id +=1 #id+1

--- Running training session 1
Epoch 1/100
      1/Unknown - 8s 8s/step - loss: 2.3026 - categorical_accuracy: 0.1406WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.136717). Check your callbacks.


704/704 [==============================] - 27s 38ms/step - loss: 2.3028 - categorical_accuracy: 0.0993 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3027 - categorical_accuracy: 0.0986 - val_loss: 2.3029 - val_categorical_accuracy: 0.0948
Epoch 3/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3027 - categorical_accuracy: 0.0994 - val_loss: 2.3029 - val_categorical_accuracy: 0.0948
Epoch 4/100
704/704 [==============================] - 18s 25ms/step - loss: 2.3027 - categorical_accuracy: 0.0981 - val_loss: 2.3032 - val_categorical_accuracy: 0.0948
Epoch 5/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3027 - categorical_accuracy: 0.0992 - val_loss: 2.3031 - val_categorical_accuracy: 0.0956
Epoch 6/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3027 - categorical_accuracy: 0.0988 - val_loss: 2.3028 - val_categorical_accurac

704/704 [==============================] - 19s 27ms/step - loss: 2.3055 - categorical_accuracy: 0.0982 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3032 - categorical_accuracy: 0.1026 - val_loss: 2.3029 - val_categorical_accuracy: 0.1088
Epoch 3/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3035 - categorical_accuracy: 0.0977 - val_loss: 2.3030 - val_categorical_accuracy: 0.1094
Epoch 4/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3037 - categorical_accuracy: 0.0988 - val_loss: 2.3030 - val_categorical_accuracy: 0.1000
Epoch 5/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3037 - categorical_accuracy: 0.0978 - val_loss: 2.3029 - val_categorical_accuracy: 0.0984
Epoch 6/100
704/704 [==============================] - 18s 25ms/step - loss: 2.3035 - categorical_accuracy: 0.0984 - val_loss: 2.3045 - val_categorical_accurac

704/704 [==============================] - 19s 27ms/step - loss: 2.7989 - categorical_accuracy: 0.1000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3054 - categorical_accuracy: 0.0987 - val_loss: 2.3056 - val_categorical_accuracy: 0.0940
Epoch 3/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3054 - categorical_accuracy: 0.0995 - val_loss: 2.3038 - val_categorical_accuracy: 0.0956
Epoch 4/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3054 - categorical_accuracy: 0.1011 - val_loss: 2.3043 - val_categorical_accuracy: 0.0948
Epoch 5/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3057 - categorical_accuracy: 0.0984 - val_loss: 2.3065 - val_categorical_accuracy: 0.0940
Epoch 6/100
704/704 [==============================] - 17s 24ms/step - loss: 2.3054 - categorical_accuracy: 0.0993 - val_loss: 2.3048 - val_categorical_accurac

704/704 [==============================] - 19s 27ms/step - loss: 1.8659 - categorical_accuracy: 0.3005 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 17s 24ms/step - loss: 1.4944 - categorical_accuracy: 0.4632 - val_loss: 1.2435 - val_categorical_accuracy: 0.5334
Epoch 3/100
704/704 [==============================] - 17s 24ms/step - loss: 1.3204 - categorical_accuracy: 0.5301 - val_loss: 1.1240 - val_categorical_accuracy: 0.6062
Epoch 4/100
704/704 [==============================] - 18s 25ms/step - loss: 1.1962 - categorical_accuracy: 0.5760 - val_loss: 1.0412 - val_categorical_accuracy: 0.6304
Epoch 5/100
704/704 [==============================] - 18s 25ms/step - loss: 1.0895 - categorical_accuracy: 0.6159 - val_loss: 0.9940 - val_categorical_accuracy: 0.6356
Epoch 6/100
704/704 [==============================] - 17s 24ms/step - loss: 1.0151 - categorical_accuracy: 0.6455 - val_loss: 0.9620 - val_categorical_accurac

...

704/704 [==============================] - 19s 27ms/step - loss: 0.0885 - categorical_accuracy: 0.9755 - val_loss: 1.1888 - val_categorical_accuracy: 0.8050
Epoch 50/100
704/704 [==============================] - 19s 27ms/step - loss: 0.0735 - categorical_accuracy: 0.9778 - val_loss: 1.3649 - val_categorical_accuracy: 0.7814
Epoch 51/100
704/704 [==============================] - 19s 27ms/step - loss: 0.0808 - categorical_accuracy: 0.9748 - val_loss: 1.2153 - val_categorical_accuracy: 0.7962
Epoch 52/100
704/704 [==============================] - 19s 27ms/step - loss: 0.0616 - categorical_accuracy: 0.9804 - val_loss: 1.4867 - val_categorical_accuracy: 0.7806
Epoch 53/100
704/704 [==============================] - 19s 27ms/step - loss: 0.0657 - categorical_accuracy: 0.9789 - val_loss: 1.1494 - val_categorical_accuracy: 0.8004
Epoch 54/100
704/704 [==============================] - 19s 27ms/step - loss: 0.0658 - categorical_accuracy: 0.9797 - val_loss: 1.1060 - val_categorical_accuracy: 

704/704 [==============================] - 19s 27ms/step - loss: 0.0441 - categorical_accuracy: 0.9863 - val_loss: 1.9404 - val_categorical_accuracy: 0.7368
Epoch 98/100
704/704 [==============================] - 19s 27ms/step - loss: 0.0399 - categorical_accuracy: 0.9869 - val_loss: 1.4950 - val_categorical_accuracy: 0.7962
Epoch 99/100
704/704 [==============================] - 19s 27ms/step - loss: 0.0527 - categorical_accuracy: 0.9850 - val_loss: 1.4421 - val_categorical_accuracy: 0.7896
Epoch 100/100
704/704 [==============================] - 19s 27ms/step - loss: 0.0436 - categorical_accuracy: 0.9879 - val_loss: 1.2817 - val_categorical_accuracy: 0.8144
--- Running training session 18
Epoch 1/100
      1/Unknown - 1s 1s/step - loss: 3.2310 - categorical_accuracy: 0.0938WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.140258). Check your callbacks.


704/704 [==============================] - 22s 31ms/step - loss: 1.7143 - categorical_accuracy: 0.3671 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 19s 27ms/step - loss: 1.3987 - categorical_accuracy: 0.5040 - val_loss: 2.0558 - val_categorical_accuracy: 0.4138
Epoch 3/100
704/704 [==============================] - 19s 27ms/step - loss: 1.2378 - categorical_accuracy: 0.5667 - val_loss: 1.1083 - val_categorical_accuracy: 0.6100
Epoch 4/100
704/704 [==============================] - 19s 27ms/step - loss: 1.1051 - categorical_accuracy: 0.6160 - val_loss: 1.2144 - val_categorical_accuracy: 0.5656
Epoch 5/100
704/704 [==============================] - 19s 27ms/step - loss: 1.0137 - categorical_accuracy: 0.6515 - val_loss: 1.3473 - val_categorical_accuracy: 0.5668
Epoch 6/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9470 - categorical_accuracy: 0.6773 - val_loss: 1.1155 - val_categorical_accurac

704/704 [==============================] - 19s 27ms/step - loss: 0.1545 - categorical_accuracy: 0.9508 - val_loss: 1.1408 - val_categorical_accuracy: 0.7900
Epoch 50/100
704/704 [==============================] - 19s 27ms/step - loss: 0.1529 - categorical_accuracy: 0.9517 - val_loss: 1.1904 - val_categorical_accuracy: 0.7850
Epoch 51/100
704/704 [==============================] - 19s 27ms/step - loss: 0.1698 - categorical_accuracy: 0.9485 - val_loss: 1.5603 - val_categorical_accuracy: 0.7718
Epoch 52/100
704/704 [==============================] - 19s 27ms/step - loss: 0.1541 - categorical_accuracy: 0.9516 - val_loss: 1.2151 - val_categorical_accuracy: 0.7944
Epoch 53/100
704/704 [==============================] - 29s 41ms/step - loss: 0.1599 - categorical_accuracy: 0.9534 - val_loss: 1.3405 - val_categorical_accuracy: 0.7842
Epoch 54/100
704/704 [==============================] - 22s 31ms/step - loss: 0.1474 - categorical_accuracy: 0.9546 - val_loss: 1.3774 - val_categorical_accuracy: 

704/704 [==============================] - 18s 25ms/step - loss: 2.3027 - categorical_accuracy: 0.0990 - val_loss: 2.3030 - val_categorical_accuracy: 0.0948
Epoch 32/100
704/704 [==============================] - 18s 25ms/step - loss: 2.3027 - categorical_accuracy: 0.0992 - val_loss: 2.3027 - val_categorical_accuracy: 0.0948
Epoch 33/100
704/704 [==============================] - 18s 25ms/step - loss: 2.3027 - categorical_accuracy: 0.0974 - val_loss: 2.3030 - val_categorical_accuracy: 0.0990
Epoch 34/100
704/704 [==============================] - 18s 25ms/step - loss: 2.3027 - categorical_accuracy: 0.0990 - val_loss: 2.3028 - val_categorical_accuracy: 0.0990
Epoch 35/100
704/704 [==============================] - 18s 25ms/step - loss: 2.3027 - categorical_accuracy: 0.0985 - val_loss: 2.3029 - val_categorical_accuracy: 0.0948
Epoch 36/100
704/704 [==============================] - 18s 26ms/step - loss: 2.3027 - categorical_accuracy: 0.0996 - val_loss: 2.3028 - val_categorical_accuracy: 